In [1]:
import numpy as np 
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import r2_score

In [12]:
data = pd.read_csv('MELBOURNE_HOUSE_PRICES_LESS.csv')

In [13]:
data

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,Abbotsford,49 Lithgow St,3,h,1490000.0,S,Jellis,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,Abbotsford,59A Turner St,3,h,1220000.0,S,Marshall,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
2,Abbotsford,119B Yarra St,3,h,1420000.0,S,Nelson,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
3,Aberfeldie,68 Vida St,3,h,1515000.0,S,Barry,1/04/2017,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,Airport West,92 Clydesdale Rd,2,h,670000.0,S,Nelson,1/04/2017,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63018,Roxburgh Park,3 Carr Pl,3,h,566000.0,S,Raine,31/03/2018,3064,Northern Metropolitan,5833,20.6,Hume City Council
63019,Roxburgh Park,9 Parker Ct,3,h,500000.0,S,Raine,31/03/2018,3064,Northern Metropolitan,5833,20.6,Hume City Council
63020,Roxburgh Park,5 Parkinson Wy,3,h,545000.0,S,Raine,31/03/2018,3064,Northern Metropolitan,5833,20.6,Hume City Council
63021,Thomastown,3/1 Travers St,3,u,NaN,PI,Barry,31/03/2018,3074,Northern Metropolitan,7955,15.3,Whittlesea City Council


In [14]:
{column: len(data[column].unique()) for column in data.columns}

{'Suburb': 380,
 'Address': 57754,
 'Rooms': 14,
 'Type': 3,
 'Price': 3418,
 'Method': 9,
 'SellerG': 476,
 'Date': 112,
 'Postcode': 225,
 'Regionname': 8,
 'Propertycount': 368,
 'Distance': 180,
 'CouncilArea': 34}

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63023 entries, 0 to 63022
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         63023 non-null  object 
 1   Address        63023 non-null  object 
 2   Rooms          63023 non-null  int64  
 3   Type           63023 non-null  object 
 4   Price          48433 non-null  float64
 5   Method         63023 non-null  object 
 6   SellerG        63023 non-null  object 
 7   Date           63023 non-null  object 
 8   Postcode       63023 non-null  int64  
 9   Regionname     63023 non-null  object 
 10  Propertycount  63023 non-null  int64  
 11  Distance       63023 non-null  float64
 12  CouncilArea    63023 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 6.3+ MB


In [16]:
def onehot_encode(df, column_dict):
    df = df.copy()
    for column, prefix in column_dict.items():
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df =pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [17]:
def get_sequences(texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    max_seq_length = np.max([len(sequence) for sequence in sequences])
    print("Max Sequence length:", max_seq_length)
    sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')
    return sequences

In [20]:
get_sequences(X['Address'])

NameError: name 'X' is not defined

In [21]:
def preprocessing_data(df):
  df =df.copy()
  df = df.dropna(axis=0).reset_index(drop=True)
  df['Date'] = pd.to_datetime(df['Date'])
  column_dict = {
 'Suburb': 'SUB',
 'Type': 'TP',
 'Method': 'ME',
 'SellerG': 'SE',
 'Postcode': 'PC',
 'Regionname': 'RG',
 'CouncilArea': 'CA',
  }
  df = onehot_encode(df, column_dict)
  df['Year'] = df['Date'].apply(lambda x: x.year)
  df['Month'] = df['Date'].apply(lambda x: x.month)
  df['Day'] = df['Date'].apply(lambda x: x.day)
  df = df.drop('Date', axis=1)
  addr_sequences = get_sequences(df['Address'])
  df = df.drop('Address', axis=1)
  
 # splitting into X and y
  y = df['Price']
  X = df.drop('Price', axis=1).copy()
  scaler = StandardScaler()
  X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
  return X, addr_sequences, y

In [22]:
X, addr_sequences, y = preprocessing_data(data)

Max Sequence length: 5


In [23]:
X

,Rooms,Propertycount,Distance,SUB_Abbotsford,SUB_Aberfeldie,SUB_Airport West,SUB_Albanvale,SUB_Albert Park,SUB_Albion,SUB_Alphington,...,CA_Port Phillip City Council,CA_Stonnington City Council,CA_Whitehorse City Council,CA_Whittlesea City Council,CA_Wyndham City Council,CA_Yarra City Council,CA_Yarra Ranges Shire Council,Year,Month,Day
0,-0.075861,-0.795852,-1.285142,18.441189,-0.044332,-0.065356,-0.020325,-0.051476,-0.043147,-0.04193,...,-0.165485,-0.161289,-0.167385,-0.210713,-0.156232,5.974250,-0.060219,-0.053884,-1.909231,-1.448953
1,-0.075861,-0.795852,-1.285142,18.441189,-0.044332,-0.065356,-0.020325,-0.051476,-0.043147,-0.04193,...,-0.165485,-0.161289,-0.167385,-0.210713,-0.156232,5.974250,-0.060219,-0.053884,-1.909231,-1.448953
2,-0.075861,-0.795852,-1.285142,18.441189,-0.044332,-0.065356,-0.020325,-0.051476,-0.043147,-0.04193,...,-0.165485,-0.161289,-0.167385,-0.210713,-0.156232,5.974250,-0.060219,-0.053884,-1.909231,-1.448953
3,-0.075861,-1.351338,-0.689112,-0.054226,22.557062,-0.065356,-0.020325,-0.051476,-0.043147,-0.04193,...,-0.165485,-0.161289,-0.167385,-0.210713,-0.156232,-0.167385,-0.060219,-0.053884,-1.909231,-1.448953
4,-1.134401,-0.920365,-0.305003,-0.054226,-0.044332,15.300708,-0.020325,-0.051476,-0.043147,-0.04193,...,-0.165485,-0.161289,-0.167385,-0.210713,-0.156232,-0.167385,-0.060219,-0.053884,-1.909231,-1.448953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48428,-1.134401,2.128749,3.350649,-0.054226,-0.044332,-0.065356,-0.020325,-0.051476,-0.043147,-0.04193,...,-0.165485,-0.161289,-0.167385,-0.210713,-0.156232,-0.167385,-0.060219,1.259546,-1.258706,1.773269
48429,-0.075861,1.572814,-0.569906,-0.054226,-0.044332,-0.065356,-0.020325,-0.051476,-0.043147,-0.04193,...,-0.165485,-0.161289,-0.167385,-0.210713,-0.156232,-0.167385,-0.060219,1.259546,-1.258706,1.773269
48430,-0.075861,-0.388885,1.045999,-0.054226,-0.044332,-0.065356,-0.020325,-0.051476,-0.043147,-0.04193,...,-0.165485,-0.161289,-0.167385,-0.210713,-0.156232,-0.167385,-0.060219,1.259546,-1.258706,1.773269
48431,-0.075861,-0.388885,1.045999,-0.054226,-0.044332,-0.065356,-0.020325,-0.051476,-0.043147,-0.04193,...,-0.165485,-0.161289,-0.167385,-0.210713,-0.156232,-0.167385,-0.060219,1.259546,-1.258706,1.773269


In [24]:
X.mean()

Rooms                           -1.191255e-16
Propertycount                   -5.046698e-17
Distance                         2.505744e-16
SUB_Abbotsford                   6.455078e-18
SUB_Aberfeldie                  -1.643111e-17
                                     ...     
CA_Yarra City Council            5.692205e-17
CA_Yarra Ranges Shire Council   -3.829035e-17
Year                            -1.181068e-13
Month                            6.572443e-17
Day                             -1.877841e-16
Length: 1069, dtype: float64

In [25]:
y.mean()

997898.2414882415

In [26]:
addr_sequences

array([[  66, 2804,    1,    0,    0],
       [2408,  418,    1,    0,    0],
       [6811,  313,    1,    0,    0],
       ...,
       [   7, 3345,   26,    0,    0],
       [  15,  779,    6,    0,    0],
       [  10, 6711,   36,    0,    0]])

In [27]:
{column: len(X[column].unique()) for column in X.columns}

{'Rooms': 14,
 'Propertycount': 359,
 'Distance': 176,
 'SUB_Abbotsford': 2,
 'SUB_Aberfeldie': 2,
 'SUB_Airport West': 2,
 'SUB_Albanvale': 2,
 'SUB_Albert Park': 2,
 'SUB_Albion': 2,
 'SUB_Alphington': 2,
 'SUB_Altona': 2,
 'SUB_Altona Meadows': 2,
 'SUB_Altona North': 2,
 'SUB_Ardeer': 2,
 'SUB_Armadale': 2,
 'SUB_Ascot Vale': 2,
 'SUB_Ashburton': 2,
 'SUB_Ashwood': 2,
 'SUB_Aspendale': 2,
 'SUB_Aspendale Gardens': 2,
 'SUB_Attwood': 2,
 'SUB_Avondale Heights': 2,
 'SUB_Bacchus Marsh': 2,
 'SUB_Balaclava': 2,
 'SUB_Balwyn': 2,
 'SUB_Balwyn North': 2,
 'SUB_Bayswater': 2,
 'SUB_Bayswater North': 2,
 'SUB_Beaconsfield': 2,
 'SUB_Beaconsfield Upper': 2,
 'SUB_Beaumaris': 2,
 'SUB_Belgrave': 2,
 'SUB_Belgrave Heights': 2,
 'SUB_Belgrave South': 2,
 'SUB_Bellfield': 2,
 'SUB_Bentleigh': 2,
 'SUB_Bentleigh East': 2,
 'SUB_Berwick': 2,
 'SUB_Black Rock': 2,
 'SUB_Blackburn': 2,
 'SUB_Blackburn North': 2,
 'SUB_Blackburn South': 2,
 'SUB_Bonbeach': 2,
 'SUB_Boronia': 2,
 'SUB_Botanic Ridge'

# Training

In [28]:
X_train, X_test, addr_train, addr_test, y_train, y_test = train_test_split(X, addr_sequences, y, test_size=0.2, random_state=120)

In [29]:
X_inputs = tf.keras.Input(shape=(X_train.shape[1],), name='X_input')
addr_inputs = tf.keras.Input(shape=(addr_train.shape[1],), name='addr_input')

#X
X_dense1 = tf.keras.layers.Dense(512, activation='relu', name='X_dense1')(X_inputs)
X_dense2 = tf.keras.layers.Dense(512, activation='relu', name='X_dense2')(X_dense1)

#addr
addr_embedding =tf.keras.layers.Embedding(
input_dim=10774,
output_dim=64,
input_length=addr_train.shape[1],
name='addr_embedding')(addr_inputs)

addr_flatten = tf.keras.layers.Flatten(name='addr_flatten')(addr_embedding)
concat = tf.keras.layers.concatenate([X_dense2, addr_flatten], name='concatenate')
outputs = tf.keras.layers.Dense(1, activation='linear')(concat)

model = tf.keras.Model(inputs=[X_inputs, addr_inputs], outputs=outputs)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 X_input (InputLayer)           [(None, 1069)]       0           []                               
                                                                                                  
 addr_input (InputLayer)        [(None, 5)]          0           []                               
                                                                                                  
 X_dense1 (Dense)               (None, 512)          547840      ['X_input[0][0]']                
                                                                                                  
 addr_embedding (Embedding)     (None, 5, 64)        689536      ['addr_input[0][0]']             
                                                                                              

In [34]:
model.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
loss='mse',
metrics=['accuracy']
)
history = model.fit(
[X_train, addr_train],
y_train,
validation_split=0.2,
batch_size=32,
epochs=50,
callbacks=[
    tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau()
]
)

Epoch 1/50
969/969 [==============================] - 24s 23ms/step - loss: 190879727616.0000 - accuracy: 0.0000e+00 - val_loss: 111041798144.0000 - val_accuracy: 0.0000e+00 - lr: 0.1000
Epoch 2/50
969/969 [==============================] - 22s 23ms/step - loss: 109885022208.0000 - accuracy: 0.0000e+00 - val_loss: 103263322112.0000 - val_accuracy: 0.0000e+00 - lr: 0.1000
Epoch 3/50
969/969 [==============================] - 24s 25ms/step - loss: 100742635520.0000 - accuracy: 0.0000e+00 - val_loss: 102067617792.0000 - val_accuracy: 0.0000e+00 - lr: 0.1000
Epoch 4/50
969/969 [==============================] - 23s 24ms/step - loss: 91315814400.0000 - accuracy: 0.0000e+00 - val_loss: 101892472832.0000 - val_accuracy: 0.0000e+00 - lr: 0.1000
Epoch 5/50
969/969 [==============================] - 23s 24ms/step - loss: 84828938240.0000 - accuracy: 0.0000e+00 - val_loss: 102011944960.0000 - val_accuracy: 0.0000e+00 - lr: 0.1000
Epoch 6/50
969/969 [==============================] - 26s 27ms/step

In [35]:
y_true = np.array(y_test)
y_pred = np.squeeze(model.predict([X_test, addr_test]))

303/303 [==============================] - 1s 3ms/step


In [37]:
y_true = np.array(y_test)
y_pred = np.squeeze(model.predict([X_test, addr_test]))

303/303 [==============================] - 1s 3ms/step


In [38]:
model_r2 = r2_score(y_true, y_pred)
print('Model R^2: {:.6f}'. format(model_r2))

Model R^2: 0.731476


In [39]:
model.save('nn_with_all_features.h5')